In [36]:
import pandas as pd
import numpy as np
import torch
import pickle

#Load Static testing and training data
with open('static_data/train.pickle','rb') as handle:
    train_indices = pickle.load(handle)

with open('static_data/test.pickle','rb') as handle:
    test_indices = pickle.load(handle)

In [39]:
df = pd.read_csv("data/chest_xray_train.csv")

In [40]:
# get all pixel columns
selected_features = list(df.columns)
selected_features.remove("file_name")
selected_features.remove("class_id")

# Set train and target
X = df[selected_features]
y = df['class_id']

# Get training and test data
X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
y_train, y_test = y[train_indices], y[test_indices]

In [41]:
X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
y_train, y_test = y[train_indices], y[test_indices]

In [46]:
train_tensor = torch.tensor(X_train.values, dtype= torch.int16)
test_tensor = torch.tensor(y_test.values, dtype = torch.int16)

In [47]:
train_tensor

tensor([[  4,   3,   3,  ...,   0,   0,   0],
        [  0,  13,  45,  ...,   0,   0,   0],
        [ 20,  20,  20,  ...,  20,  20,  20],
        ...,
        [ 92, 102, 179,  ...,  23,  24,  23],
        [ 52,  65,  73,  ...,   0,   0,   0],
        [ 10,  23,  62,  ...,  24,  26,  37]], dtype=torch.int16)

In [48]:
train_tensor.shape

torch.Size([4205, 4096])

In [49]:
test_tensor

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 0, 2,
        1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 0, 2, 0, 2, 2, 2, 2, 1,
        1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        2, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1,
        2, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 2, 2, 1,
        1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 0, 2, 0, 1, 2, 2, 2, 1,
        0, 1, 1, 1, 1, 0, 2, 2, 1, 0, 0, 2, 0, 0, 1, 1, 2, 2, 2, 1, 0, 1, 0, 0,
        1, 0, 2, 0, 2, 0, 0, 0, 1, 2, 1,

In [50]:
test_tensor.shape

torch.Size([467])

In [ ]:
#Custom dataset

In [51]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms.functional import to_pil_image, pil_to_tensor
import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np
from torchvision.io import read_image

from PIL import Image
from PIL import ImageOps

class ChestxrayImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=pil_transform_tensor, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [52]:
img_dir = "C:\\Users\\conor\\OneDrive\\Desktop\\school\\Data Science Minor Independent Study\\chest-xray-research\\archive\\train_images\\train_images"
annotations_file = "C:\\Users\\conor\\OneDrive\\Desktop\\school\\Data Science Minor Independent Study\\chest-xray-research\\archive\\labels_train.csv"

In [56]:
chest_dataset = ChestxrayImageDataset(annotations_file=annotations_file, img_dir=img_dir)

In [58]:
chest_dataset.__getitem__(2)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [1]:
# PIL method
test_img = chest_dataset.__getitem__(2) # get second image in labels list
test_img[0][0].shape # access image in tensor
test_img = test_img[0][0] # image
test_img.shape
test_img = np.array(test_img) # turn to np array
test_pil_img = to_pil_image(test_img) # turn to pil and rescale
img_gray = ImageOps.grayscale(test_pil_img)
image_resized = img_gray.resize((64,64), Image.LANCZOS)
image_resized
image_resized_tensor = pil_to_tensor(image_resized) # turn back into tensor
image_resized_tensor
image_resized_tensor[0]
# plt.figure(figsize=(10,10))
# plt.imshow(image_resized_tensor[0])
# plt.show()

NameError: name 'chest_dataset' is not defined

In [55]:
def pil_transform_tensor(tensor_img):
    # test_img = chest_dataset.__getitem__(2) # get second image in labels list
    # test_img[0][0].shape # access image in tensor
    test_img = test_img[0][0] # image
    # test_img.shape
    test_img = np.array(test_img) # turn to np array
    test_pil_img = to_pil_image(test_img) # turn to pil and rescale
    img_gray = ImageOps.grayscale(test_pil_img)
    image_resized = img_gray.resize((64,64), Image.LANCZOS)
    # image_resized
    image_resized_tensor = pil_to_tensor(image_resized) # turn back into tensor
    # image_resized_tensor
    return image_resized_tensor

torch.Size([1, 64, 64])